# Skip-gram word2vec

In this notebook, I'll lead you through using TensorFlow to implement the word2vec algorithm using the skip-gram architecture. By implementing this, you'll learn about embedding words for use in natural language processing. This will come in handy when dealing with things like machine translation.

## Readings

Here are the resources I used to build this notebook. I suggest reading these either beforehand or while you're working on this material.

* A really good [conceptual overview](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/) of word2vec from Chris McCormick 
* [First word2vec paper](https://arxiv.org/pdf/1301.3781.pdf) from Mikolov et al.
* [NIPS paper](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) with improvements for word2vec also from Mikolov et al.
* An [implementation of word2vec](http://www.thushv.com/natural_language_processing/word2vec-part-1-nlp-with-deep-learning-with-tensorflow-skip-gram/) from Thushan Ganegedara
* TensorFlow [word2vec tutorial](https://www.tensorflow.org/tutorials/word2vec)

## Word embeddings

When you're dealing with words in text, you end up with tens of thousands of classes to predict, one for each word. Trying to one-hot encode these words is massively inefficient, you'll have one element set to 1 and the other 50,000 set to 0. The matrix multiplication going into the first hidden layer will have almost all of the resulting values be zero. This a huge waste of computation. 

![one-hot encodings](assets/one_hot_encoding.png)

To solve this problem and greatly increase the efficiency of our networks, we use what are called embeddings. Embeddings are just a fully connected layer like you've seen before. We call this layer the embedding layer and the weights are embedding weights. We skip the multiplication into the embedding layer by instead directly grabbing the hidden layer values from the weight matrix. We can do this because the multiplication of a one-hot encoded vector with a matrix returns the row of the matrix corresponding the index of the "on" input unit.

![lookup](assets/lookup_matrix.png)

Instead of doing the matrix multiplication, we use the weight matrix as a lookup table. We encode the words as integers, for example "heart" is encoded as 958, "mind" as 18094. Then to get hidden layer values for "heart", you just take the 958th row of the embedding matrix. This process is called an **embedding lookup** and the number of hidden units is the **embedding dimension**.

<img src='assets/tokenize_lookup.png' width=500>
 
There is nothing magical going on here. The embedding lookup table is just a weight matrix. The embedding layer is just a hidden layer. The lookup is just a shortcut for the matrix multiplication. The lookup table is trained just like any weight matrix as well.

Embeddings aren't only used for words of course. You can use them for any model where you have a massive number of classes. A particular type of model called **Word2Vec** uses the embedding layer to find vector representations of words that contain semantic meaning.



## Word2Vec

The word2vec algorithm finds much more efficient representations by finding vectors that represent the words. These vectors also contain semantic information about the words. Words that show up in similar contexts, such as "black", "white", and "red" will have vectors near each other. There are two architectures for implementing word2vec, CBOW (Continuous Bag-Of-Words) and Skip-gram.

<img src="assets/word2vec_architectures.png" width="500">

In this implementation, we'll be using the skip-gram architecture because it performs better than CBOW. Here, we pass in a word and try to predict the words surrounding it in the text. In this way, we can train the network to learn representations for words that show up in similar contexts.

First up, importing packages.

In [1]:
import time

import numpy as np
import tensorflow as tf

import utils

Load the [text8 dataset](http://mattmahoney.net/dc/textdata.html), a file of cleaned up Wikipedia articles from Matt Mahoney. The next cell will download the data set to the `data` folder. Then you can extract it and delete the archive file to save storage space.

In [2]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm
import zipfile

dataset_folder_path = 'data'
dataset_filename = 'text8.zip'
dataset_name = 'Text8 Dataset'

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile(dataset_filename):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc=dataset_name) as pbar:
        urlretrieve(
            'http://mattmahoney.net/dc/text8.zip',
            dataset_filename,
            pbar.hook)

if not isdir(dataset_folder_path):
    with zipfile.ZipFile(dataset_filename) as zip_ref:
        zip_ref.extractall(dataset_folder_path)
        
with open('data/text8') as f:
    text = f.read()

Text8 Dataset: 31.4MB [00:23, 1.33MB/s]                            


## Preprocessing

Here I'm fixing up the text to make training easier. This comes from the `utils` module I wrote. The `preprocess` function coverts any punctuation into tokens, so a period is changed to ` <PERIOD> `. In this data set, there aren't any periods, but it will help in other NLP problems. I'm also removing all words that show up five or fewer times in the dataset. This will greatly reduce issues due to noise in the data and improve the quality of the vector representations. If you want to write your own functions for this stuff, go for it.

In [3]:
words = utils.preprocess(text)
print(words[:30])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst']


In [4]:
print("Total words: {}".format(len(words)))
print("Unique words: {}".format(len(set(words))))

Total words: 16680599
Unique words: 63641


And here I'm creating dictionaries to convert words to integers and backwards, integers to words. The integers are assigned in descending frequency order, so the most frequent word ("the") is given the integer 0 and the next most frequent is 1 and so on. The words are converted to integers and stored in the list `int_words`.

In [5]:
vocab_to_int, int_to_vocab = utils.create_lookup_tables(words)
int_words = [vocab_to_int[word] for word in words]

In [11]:
print(int_to_vocab[0])
print(vocab_to_int['the'])
int_words[0]

the
0


5240

## Subsampling

Words that show up often such as "the", "of", and "for" don't provide much context to the nearby words. If we discard some of them, we can remove some of the noise from our data and in return get faster training and better representations. This process is called subsampling by Mikolov. For each word $w_i$ in the training set, we'll discard it with probability given by 

$$ P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}} $$

where $t$ is a threshold parameter and $f(w_i)$ is the frequency of word $w_i$ in the total dataset.

I'm going to leave this up to you as an exercise. This is more of a programming challenge, than about deep learning specifically. But, being able to prepare your data for your network is an important skill to have. Check out my solution to see how I did it.

> **Exercise:** Implement subsampling for the words in `int_words`. That is, go through `int_words` and discard each word given the probablility $P(w_i)$ shown above. Note that $P(w_i)$ is the probability that a word is discarded. Assign the subsampled data to `train_words`.

In [12]:
from collections import Counter

freq = Counter(int_words)

Counter({0: 1061396,
         1: 593677,
         2: 416629,
         3: 411764,
         4: 372201,
         5: 325873,
         6: 316376,
         7: 264975,
         8: 250430,
         9: 192644,
         10: 183153,
         11: 131815,
         12: 125285,
         13: 118445,
         14: 116710,
         15: 115789,
         16: 114775,
         17: 112807,
         18: 111831,
         19: 109510,
         20: 108182,
         21: 102145,
         22: 99683,
         23: 95603,
         24: 91250,
         25: 76527,
         26: 73334,
         27: 72871,
         28: 68945,
         29: 62603,
         30: 61925,
         31: 61281,
         32: 58832,
         33: 54788,
         34: 54576,
         35: 53573,
         36: 44358,
         37: 44033,
         38: 39712,
         39: 39086,
         40: 37866,
         41: 35358,
         42: 32433,
         43: 31523,
         44: 29567,
         45: 28810,
         46: 28553,
         47: 28161,
         48: 28100,
       

In [18]:
threshold = 1e-5
for word in int_words:
    probability = 1 - np.sqrt((threshold/freq[word]))
    print(str(word)+":"+ str(probability))
    #train_words = # The final subsampled word list


5240:0.999818331895
3081:0.999867778529
11:0.999991290011
5:0.999994460433
194:0.999962781276
1:0.999995895834
3133:0.999866725886
45:0.999981369334
58:0.99997902831
155:0.999965562255
127:0.999968645721
741:0.999933642283
476:0.999945862838
10583:0.99970638989
133:0.999967780469
0:0.999996930545
27585:0.999367544468
1:0.999995895834
0:0.999996930545
102:0.999970972393
854:0.999929796466
2:0.999995100799
0:0.999996930545
15183:0.999616517506
58576:0.998709005551
1:0.999995895834
0:0.999996930545
150:0.999966166752
854:0.999929796466
3584:0.99985581255
0:0.999996930545
194:0.999962781276
10:0.999992610876
190:0.999963184502
58:0.99997902831
4:0.999994816642
5:0.999994460433
10756:0.999703825561
214:0.999960573046
6:0.999994377904
1325:0.999913997385
104:0.999970892574
454:0.999946563017
19:0.999990444067
58:0.99997902831
2733:0.999876250512
362:0.999951000404
6:0.999994377904
3673:0.999853510316
0:0.999996930545
708:0.999935097766
1:0.999995895834
371:0.999950413562
26:0.999988322569
40

193:0.99996289163
85:0.999973733278
47:0.999981155876
3235:0.999863791059
241:0.999958498721
1896:0.999897991043
6795:0.999784334454
133:0.999967780469
5240:0.999818331895
230:0.999959364363
6:0.999994377904
44:0.99998160938
6695:0.999786313078
158:0.999965314558
6:0.999994377904
1022:0.999924255465
1555:0.999906911272
1818:0.99990009985
0:0.999996930545
193:0.99996289163
134:0.999967709999
27:0.99998828553
0:0.999996930545
1112:0.999921066296
2771:0.999875097542
2980:0.999870140821
17:0.999990584747
5:0.999994460433
801:0.999931222322
1231:0.99991695452
4:0.999994816642
0:0.999996930545
193:0.99996289163
35:0.999986337599
17:0.999990584747
669:0.999936665717
6:0.999994377904
331:0.999952574134
10988:0.999699849887
152:0.999966016655
481:0.999945775359
1:0.999995895834
0:0.999996930545
756:0.999933050056
0:0.999996930545
45:0.999981369334
8954:0.999739179735
6:0.999994377904
2980:0.999870140821
421:0.999947998419
27:0.99998828553
0:0.999996930545
56:0.99997958631
1346:0.999913223733
32

1:0.999995895834
0:0.999996930545
208:0.999961086843
2:0.999995100799
629:0.999938926671
1:0.999995895834
0:0.999996930545
1411:0.999911053992
1:0.999995895834
867:0.999929447886
4:0.999994816642
302:0.999954418137
99:0.999971541024
0:0.999996930545
18900:0.999543564535
421:0.999947998419
689:0.999935968437
6:0.999994377904
12716:0.999664799238
4:0.999994816642
0:0.999996930545
595:0.999939926666
3:0.999995071942
8:0.999993680877
16:0.999990665816
20:0.999990385593
6987:0.999780206509
7088:0.999778051619
909:0.999928018425
79:0.999974284476
5:0.999994460433
85:0.999973733278
927:0.999927528621
738:0.999933744316
4:0.999994816642
803:0.999931140842
0:0.999996930545
30147:0.999309934441
1448:0.999910161116
1931:0.999896912694
6:0.999994377904
2521:0.999881654733
5:0.999994460433
234:0.999959062437
927:0.999927528621
3264:0.99986302655
2007:0.999894998294
2:0.999995100799
45147:0.999
18:0.999990543751
30147:0.999309934441
2029:0.999894532135
280:0.999955753945
6:0.999994377904
5:0.9999944

7088:0.999778051619
1693:0.999903145144
19:0.999990444067
0:0.999996930545
93:0.999972162013
5830:0.999805375264
6:0.999994377904
985:0.99992560891
5:0.999994460433
3558:0.999856408368
24:0.999989531522
1666:0.999903863929
2:0.999995100799
553:0.999941905613
1666:0.999903863929
6:0.999994377904
2601:0.999879788241
4326:0.9998382038
2258:0.99988861352
148:0.999966332066
1736:0.999901989042
40:0.99998374918
53:0.999980151462
2272:0.999888056584
6:0.999994377904
7433:0.999770584266
310:0.999954005363
4792:0.999827739759
2215:0.999889836838
73:0.999976010975
3848:0.999848900529
33145:0.999254644008
67:0.999977181792
36:0.999984985391
5240:0.999818331895
2:0.999995100799
371:0.999950413562
462:0.999946107108
445:0.999946895584
2:0.999995100799
813:0.999931009868
3416:0.999859280491
335:0.999952483203
5240:0.999818331895
13699:0.999644215967
0:0.999996930545
1002:0.999924962472
1:0.999995895834
5:0.999994460433
93:0.999972162013
2752:0.999875774001
2:0.999995100799
29282:0.999340619527
2:0.9

2353:0.999885816772
2:0.999995100799
447:0.999946820546
1311:0.999914313689
222:0.999959961545
0:0.999996930545
683:0.99993619043
439:0.999947155738
47:0.999981155876
382:0.99994986193
23:0.999989772624
7683:0.999765596385
38:0.999984131381
2320:0.999886699645
43:0.999982189085
345:0.999951814922
2:0.999995100799
42:0.999982440731
2825:0.999873811384
6:0.999994377904
0:0.999996930545
228:0.99995964657
99:0.999971541024
46:0.999981285677
52:0.999980204423
1447:0.999910197349
3921:0.99984714554
4:0.999994816642
2424:0.999884220789
601:0.999939796166
2:0.999995100799
345:0.999951814922
634:0.999938570488
41:0.999983182705
59:0.999979014461
25:0.999988568778
23697:0.999449518117
2641:0.99987891013
19:0.999990444067
30:0.999987292299
2277:0.999887845569
7668:0.999766237709
27:0.99998828553
7683:0.999765596385
10:0.999992610876
2181:0.99989063034
23:0.999989772624
387:0.999949716354
545:0.999942427864
76:0.999974890694
55:0.999979628288
11577:0.999686887854
382:0.99994986193
2:0.999995100799

...

29:0.999987361299
22162:0.999472953723
1:0.999995895834
0:0.999996930545
1772:0.999901130535
3:0.999995071942
8:0.999993680877
21:0.999990105555
12:0.999991065907
498:0.999944976817
23:0.999989772624
55:0.999979628288
1769:0.999901178823
3252:0.999863410409
86:0.999973681476
311:0.99995399076
60113:0.998709005551
5:0.999994460433
208:0.999961086843
1:0.999995895834
450:0.99994672252
8377:0.999750777607
77:0.999974791963
225:0.999959787112
14400:0.999629883395
24:0.999989531522
329:0.999952632694
2:0.999995100799
50:0.999980471953
15271:0.999613666295
1275:0.999915544878
8386:0.999750777607
31961:0.99927452375
32:0.999986962534
208:0.999961086843
86:0.999973681476
11:0.999991290011
208:0.999961086843
10:0.999992610876
37:0.999984930083
3418:0.999859140958
74:0.999975913053
14:0.999990743518
1344:0.999913289003
215:0.999960530074
0:0.999996930545
695:0.999935423365
9231:0.999733688179
23:0.999989772624
118:0.999969327174
311:0.99995399076
1:0.999995895834
32:0.999986962534
543:0.99994252

141:0.999967183999
3644:0.999854135009
39:0.999984004811
34:0.999986463726
253:0.999957674209
4989:0.999824046164
77:0.999974791963
0:0.999996930545
439:0.999947155738
2:0.999995100799
926:0.999927528621
46:0.999981285677
39:0.999984004811
643:0.999938208
6:0.999994377904
1804:0.999900348173
1141:0.999920038372
453:0.999946570645
677:0.999936345748
3474:0.999858009541
754:0.999933124953
67:0.999977181792
36:0.999984985391
178:0.9999638968
1:0.999995895834
653:0.999937426629
4480:0.999834705099
5550:0.999811689106
178:0.9999638968
1:0.999995895834
4480:0.999834705099
1537:0.999907312
10421:0.999710114482
18:0.999990543751
185:0.999963438823
178:0.9999638968
1:0.999995895834
10876:0.999701192848
2559:0.999880732439
6415:0.9997923863
178:0.9999638968
1:0.999995895834
4480:0.999834705099
2392:0.999884989073
14149:0.999637261875
170:0.999964335718
158:0.999965314558
0:0.999996930545
9724:0.999723710518
1:0.999995895834
6415:0.9997923863
150:0.999966166752
6415:0.9997923863
6415:0.9997923863

33:0.99998648994
35:0.999986337599
10:0.999992610876
419:0.999948068587
67:0.999977181792
3502:0.999857433513
2264:0.999888405645
14:0.999990743518
9861:0.999721576977
732:0.999934033265
42:0.999982440731
10062:0.999717157288
15006:0.999619306506
1:0.999995895834
0:0.999996930545
289:0.999955323295
70:0.999976396312
25:0.999988568778
3111:0.999867313777
5979:0.999802357646
780:0.999932084272
746:0.999933407284
10436:0.999710114482
22732:0.999465477516
25209:0.99941277978
746:0.999933407284
16435:0.99959175171
2:0.999995100799
0:0.999996930545
2419:0.999884298311
1:0.999995895834
7305:0.999773544593
291:0.999955296518
4:0.999994816642
0:0.999996930545
3:0.999995071942
12:0.999991065907
90:0.999972661672
122:0.999969212546
23:0.999989772624
0:0.999996930545
975:0.999925936085
1:0.999995895834
173:0.999964171282
1898:0.999897937927
33:0.99998648994
893:0.99992835176
5:0.999994460433
50:0.999980471953
3395:0.999859834507
2:0.999995100799
4503:0.999834251614
3572:0.999856111384
13:0.9999908

28:0.999987956614
1913:0.999897509992
2646:0.999878821257
1:0.999995895834
14688:0.999624706687
37:0.999984930083
0:0.999996930545
19293:0.999533747596
701:0.999935301841
7305:0.999773544593
401:0.999948734798
10:0.999992610876
86:0.999973681476
4:0.999994816642
0:0.999996930545
236:0.999958697428
78:0.999974660391
0:0.999996930545
368:0.999950697623
2:0.999995100799
95:0.999971853869
1:0.999995895834
701:0.999935301841
7305:0.999773544593
30:0.999987292299
3:0.999995071942
3:0.999995071942
90:0.999972661672
122:0.999969212546
1892:0.999898044076
15669:0.999604715292
322:0.999953124428
418:0.999948110552
36925:0.999183503419
1282:0.99991515058
19:0.999990444067
46:0.999981285677
38:0.999984131381
5:0.999994460433
261:0.999956620032
295:0.999955003318
6:0.999994377904
7305:0.999773544593
33:0.99998648994
10:0.999992610876
1086:0.999921984442
8441:0.999749215069
6:0.999994377904
114:0.99997023203
33:0.99998648994
10:0.999992610876
88:0.99997341685
15694:0.999604715292
26:0.999988322569
3

KeyboardInterrupt: 

## Making batches

Now that our data is in good shape, we need to get it into the proper form to pass it into our network. With the skip-gram architecture, for each word in the text, we want to grab all the words in a window around that word, with size $C$. 

From [Mikolov et al.](https://arxiv.org/pdf/1301.3781.pdf): 

"Since the more distant words are usually less related to the current word than those close to it, we give less weight to the distant words by sampling less from those words in our training examples... If we choose $C = 5$, for each training word we will select randomly a number $R$ in range $< 1; C >$, and then use $R$ words from history and $R$ words from the future of the current word as correct labels."

> **Exercise:** Implement a function `get_target` that receives a list of words, an index, and a window size, then returns a list of words in the window around the index. Make sure to use the algorithm described above, where you choose a random number of words from the window.

In [ ]:
def get_target(words, idx, window_size=5):
    ''' Get a list of words in a window around an index. '''
    
    # Your code here
    
    return

Here's a function that returns batches for our network. The idea is that it grabs `batch_size` words from a words list. Then for each of those words, it gets the target words in the window. I haven't found a way to pass in a random number of target words and get it to work with the architecture, so I make one row per input-target pair. This is a generator function by the way, helps save memory.

In [ ]:
def get_batches(words, batch_size, window_size=5):
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    
    n_batches = len(words)//batch_size
    
    # only full batches
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y
    

## Building the graph

From [Chris McCormick's blog](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/), we can see the general structure of our network.
![embedding_network](./assets/skip_gram_net_arch.png)

The input words are passed in as integers. This will go into a hidden layer of linear units, then into a softmax layer. We'll use the softmax layer to make a prediction like normal.

The idea here is to train the hidden layer weight matrix to find efficient representations for our words. We can discard the softmax layer becuase we don't really care about making predictions with this network. We just want the embedding matrix so we can use it in other networks we build from the dataset.

I'm going to have you build the graph in stages now. First off, creating the `inputs` and `labels` placeholders like normal.

> **Exercise:** Assign `inputs` and `labels` using `tf.placeholder`. We're going to be passing in integers, so set the data types to `tf.int32`. The batches we're passing in will have varying sizes, so set the batch sizes to [`None`]. To make things work later, you'll need to set the second dimension of `labels` to `None` or `1`.

In [ ]:
train_graph = tf.Graph()
with train_graph.as_default():
    inputs = 
    labels = 

## Embedding



The embedding matrix has a size of the number of words by the number of units in the hidden layer. So, if you have 10,000 words and 300 hidden units, the matrix will have size $10,000 \times 300$. Remember that we're using tokenized data for our inputs, usually as integers, where the number of tokens is the number of words in our vocabulary.


> **Exercise:** Tensorflow provides a convenient function [`tf.nn.embedding_lookup`](https://www.tensorflow.org/api_docs/python/tf/nn/embedding_lookup) that does this lookup for us. You pass in the embedding matrix and a tensor of integers, then it returns rows in the matrix corresponding to those integers. Below, set the number of embedding features you'll use (200 is a good start), create the embedding matrix variable, and use `tf.nn.embedding_lookup` to get the embedding tensors. For the embedding matrix, I suggest you initialize it with a uniform random numbers between -1 and 1 using [tf.random_uniform](https://www.tensorflow.org/api_docs/python/tf/random_uniform).

In [ ]:
n_vocab = len(int_to_vocab)
n_embedding =  # Number of embedding features 
with train_graph.as_default():
    embedding = # create embedding weight matrix here
    embed = # use tf.nn.embedding_lookup to get the hidden layer output

## Negative sampling



For every example we give the network, we train it using the output from the softmax layer. That means for each input, we're making very small changes to millions of weights even though we only have one true example. This makes training the network very inefficient. We can approximate the loss from the softmax layer by only updating a small subset of all the weights at once. We'll update the weights for the correct label, but only a small number of incorrect labels. This is called ["negative sampling"](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf). Tensorflow has a convenient function to do this, [`tf.nn.sampled_softmax_loss`](https://www.tensorflow.org/api_docs/python/tf/nn/sampled_softmax_loss).

> **Exercise:** Below, create weights and biases for the softmax layer. Then, use [`tf.nn.sampled_softmax_loss`](https://www.tensorflow.org/api_docs/python/tf/nn/sampled_softmax_loss) to calculate the loss. Be sure to read the documentation to figure out how it works.

In [ ]:
# Number of negative labels to sample
n_sampled = 100
with train_graph.as_default():
    softmax_w = # create softmax weight matrix here
    softmax_b = # create softmax biases here
    
    # Calculate the loss using negative sampling
    loss = tf.nn.sampled_softmax_loss 
    
    cost = tf.reduce_mean(loss)
    optimizer = tf.train.AdamOptimizer().minimize(cost)

## Validation

This code is from Thushan Ganegedara's implementation. Here we're going to choose a few common words and few uncommon words. Then, we'll print out the closest words to them. It's a nice way to check that our embedding table is grouping together words with similar semantic meanings.

In [ ]:
with train_graph.as_default():
    ## From Thushan Ganegedara's implementation
    valid_size = 16 # Random set of words to evaluate similarity on.
    valid_window = 100
    # pick 8 samples from (0,100) and (1000,1100) each ranges. lower id implies more frequent 
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples, 
                               random.sample(range(1000,1000+valid_window), valid_size//2))

    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keep_dims=True))
    normalized_embedding = embedding / norm
    valid_embedding = tf.nn.embedding_lookup(normalized_embedding, valid_dataset)
    similarity = tf.matmul(valid_embedding, tf.transpose(normalized_embedding))

In [ ]:
# If the checkpoints directory doesn't exist:
!mkdir checkpoints

## Training

Below is the code to train the network. Every 100 batches it reports the training loss. Every 1000 batches, it'll print out the validation words.

In [ ]:
epochs = 10
batch_size = 1000
window_size = 10

with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    iteration = 1
    loss = 0
    sess.run(tf.global_variables_initializer())

    for e in range(1, epochs+1):
        batches = get_batches(train_words, batch_size, window_size)
        start = time.time()
        for x, y in batches:
            
            feed = {inputs: x,
                    labels: np.array(y)[:, None]}
            train_loss, _ = sess.run([cost, optimizer], feed_dict=feed)
            
            loss += train_loss
            
            if iteration % 100 == 0: 
                end = time.time()
                print("Epoch {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Avg. Training loss: {:.4f}".format(loss/100),
                      "{:.4f} sec/batch".format((end-start)/100))
                loss = 0
                start = time.time()
            
            if iteration % 1000 == 0:
                ## From Thushan Ganegedara's implementation
                # note that this is expensive (~20% slowdown if computed every 500 steps)
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = int_to_vocab[valid_examples[i]]
                    top_k = 8 # number of nearest neighbors
                    nearest = (-sim[i, :]).argsort()[1:top_k+1]
                    log = 'Nearest to %s:' % valid_word
                    for k in range(top_k):
                        close_word = int_to_vocab[nearest[k]]
                        log = '%s %s,' % (log, close_word)
                    print(log)
            
            iteration += 1
    save_path = saver.save(sess, "checkpoints/text8.ckpt")
    embed_mat = sess.run(normalized_embedding)

Restore the trained network if you need to:

In [ ]:
with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    embed_mat = sess.run(embedding)

## Visualizing the word vectors

Below we'll use T-SNE to visualize how our high-dimensional word vectors cluster together. T-SNE is used to project these vectors into two dimensions while preserving local stucture. Check out [this post from Christopher Olah](http://colah.github.io/posts/2014-10-Visualizing-MNIST/) to learn more about T-SNE and other ways to visualize high-dimensional data.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [ ]:
viz_words = 500
tsne = TSNE()
embed_tsne = tsne.fit_transform(embed_mat[:viz_words, :])

In [ ]:
fig, ax = plt.subplots(figsize=(14, 14))
for idx in range(viz_words):
    plt.scatter(*embed_tsne[idx, :], color='steelblue')
    plt.annotate(int_to_vocab[idx], (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=0.7)